In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`
import $ivy.`org.apache.spark::spark-mllib:2.4.5`
import $ivy.`org.apache.spark::spark-streaming-kafka-0-10:2.4.5`
import $ivy.`org.apache.spark::spark-sql-kafka-0-10:2.4.5`

import $ivy.$                                  

import $ivy.$                                    

import $ivy.$                                                   

import $ivy.$                                             

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.SparkSession

def killAll() = {
    SparkSession
        .active
        .streams
        .active
        .foreach { x =>
                    val desc = x.lastProgress.sources.head.description
                    x.stop
                    println(s"Stopped ${desc}")
        }               
}

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


import org.apache.spark.sql.streaming.Trigger

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.SparkSession


defined function killAll

In [3]:
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .config("spark.executor.instances", "2")
    .config("spark.executor.memory", "2g")
    .getOrCreate()
}

import spark.implicits._

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@22acac0d
import spark.implicits._

In [4]:
val df_feedback_test = spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "172.27.1.16:9092")
  .option("subscribe", "feedback_topic_test")
//   .option("startingOffsets", """ { "feedback_topic": { "0": 1 } } """)
//   .option("endingOffsets",   """ { "feedback_topic": { "0": 100 } }  """)
   .option("enable.auto.commit", "false")
   .option("auto.offset.reset", "earliest")
   .option("startingOffsets", "earliest")
//   .option("endingOffsets", "latest")
  .load()
  .select(split($"key", "_").getItem(0) as "useID", split($"key", "_").getItem(1) as "objID", $"value" cast "string")

def createConsoleSink(df: DataFrame) = {
    df
    .writeStream
    .format("console")
    .trigger(Trigger.ProcessingTime("2 seconds"))
    .option("truncate", "false")
    .option("numRows", "10")
}

val sink = createConsoleSink(df_feedback_test)
val sq = sink.start()

df_feedback_test: DataFrame = [useID: string, objID: string ... 1 more field]
defined function createConsoleSink
sink: streaming.DataStreamWriter[Row] = org.apache.spark.sql.streaming.DataStreamWriter@3561177c
sq: streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@6d151707

In [ ]:
sq.awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+-----+-----+
|useID|objID|value|
+-----+-----+-----+
+-----+-----+-----+



start at cmd3.sc:25

1 / 1

-------------------------------------------
Batch: 1
-------------------------------------------
+--------+-------+---------------------------------------------------------------------+
|useID   |objID  |value                                                                |
+--------+-------+---------------------------------------------------------------------+
|11700150|1536025|{"membership_status":"A","feedback":["Ignored"]}                     |
|13798707|330    |{"membership_status":"U","feedback":["Liked"]}                       |
|2820138 |18010  |{"membership_status":"U","feedback":["Disliked","Clicked","Ignored"]}|
|14653794|6305   |{"membership_status":"U","feedback":["Disliked","Ignored"]}          |
|14277726|1241   |{"membership_status":"U","feedback":["Ignored"]}                     |
|7559148 |6005   |{"membership_status":"A","feedback":["Clicked","Liked"]}             |
|10716810|330    |{"membership_status":"P","feedback":["Clicked","Unliked"]}           |
|12567543|814

In [ ]:
killAll()

cmd0.sc:1: not found: value killAll
val res0 = killAll()
           ^Compilation Failed

: 

In [ ]:
spark.catalog.clearCache()
spark.close()
spark.stop()